In [1]:
!pip3 install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 588.3 MB 16 kB/s 
     |████████████████████████████████| 439 kB 57.1 MB/s 
     |████████████████████████████████| 6.0 MB 45.2 MB/s 
     |████████████████████████████████| 1.7 MB 48.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import tensorflow_hub as hub
warnings.filterwarnings('ignore')


In [3]:
from google.colab import drive, files
drive.mount('/content/drive')
!cp /content/drive/MyDrive/MLCYBER/Project/load_dataset.py .
!cp /content/drive/MyDrive/MLCYBER/Project/load_model.py .

Mounted at /content/drive


In [4]:
%run load_dataset.py
%run load_model.py

### Dataset

download kaggle dataset: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/download?datasetVersionNumber=1  
download liar dataset: https://www.cs.ucsb.edu/~william/data/liar_dataset.zip

In [5]:
import keras
from load_dataset import kaggle_dataset
from load_dataset import liar_dataset
# Select a model
from load_model import lstm
# lstm(LSTM_units, embedding)
from load_model import cnn_lstm
# cnn_lstm(cov_filters, cov_kernel, pool_size, LSTM_units, embedding)
from load_model import dense
# dense(embedding, neural=64)
from load_model import dense_dropout
# dense_dropout(embedding, dropout=0.1, neural_1=256, neural_2=64)
kaggle_path = "/content/drive/MyDrive/MLCYBER/Data/Fake-News/"
liar_path = "/content/drive/MyDrive/MLCYBER/Data/LIAR/"

# Pick dataset you want:
# If kaggle:
# dataset = kaggle_dataset(kaggle_path)
# If Liar:
dataset = liar_dataset(liar_path)

train_sentences = dataset.train_sentences
val_sentences = dataset.val_sentences
train_labels = dataset.train_labels
val_labels = dataset.val_labels

In [6]:
print(val_sentences[30])

Its been since 1888 that a Senate of a different party than the president in the White House confirmed a Supreme Court nominee.


In [ ]:
# for the fake news detection
import csv
with open("/content/drive/MyDrive/MLCYBER/Data/Fake-News/test.csv", 'wt') as out_file:
    tsv_writer = csv.writer(out_file)
    for i in range(len(val_sentences)):
        tsv_writer.writerow([val_sentences[i], val_labels[i]])

### Model

##### Word Enbedding:  
pre-trained text embedding model from TensorFlow Hub  
more text-embedding download in https://tfhub.dev/s?module-type=text-embedding  
NNLM:  
plain text -> value [batch_size, 128]  
BERT:  
plain text -> pooled_output [batch_size, 128]  
              sequence_output [batch_size, seq_length, 128]  

##### Network

In [ ]:
# Split a small batch of dataset to continue (save time...)
i = 2000
train_x = train_sentences[:i]
val_x = val_sentences[:int(0.2*i)]
train_y = train_labels[:i]
val_y = val_labels[:int(0.2*i)]

In [ ]:
cov_filters = 64
cov_kernel = 5
pool_size = 4
LSTM_units = 100
dropout = 0.2
embedding = "bert"
neural1 = 256 #for dense
neural2 = 64 #for dense

In [ ]:
training = True
#training = False
epochs = 5
name = 'cnn_lstm'

if training:
    if name == 'lstm':
        model = lstm(LSTM_units = LSTM_units, dropout = dropout, embedding = embedding)
    elif name == 'cnn_lstm':
        model = cnn_lstm(cov_filters = cov_filters, cov_kernel = cov_kernel, pool_size = pool_size, 
                         LSTM_units = LSTM_units, dropout = dropout, embedding = embedding)
    elif name == 'dense':
        model = dense(embedding = embedding, neural = neural2)
    elif name == 'dense_dropout':
        model = dense_dropout(embedding = embedding, dropout = dropout, neural_1 = neural1, neural_2 = neural2)
    else:
        print("Invalid model name")
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_history = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs=epochs)
    model.save(f"/content/drive/MyDrive/MLCYBER/Models/model_{name}_{embedding}.h5")
else:
    model = keras.models.load_model(f"/content/drive/MyDrive/MLCYBER/Models/model_{name}_{embedding}.h5", custom_objects={'KerasLayer': hub.KerasLayer})
    print("model loaded")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

### Attack (TBC)

In [7]:
#read attacked datas
df_neg_attacked = pd.read_csv("/content/drive/MyDrive/MLCYBER/Data/LIAR/test_neg_attacked.tsv", sep="\t", header=None)
df_neg_unattacked = pd.read_csv("/content/drive/MyDrive/MLCYBER/Data/LIAR/test_neg_unattacked.tsv", sep="\t", header=None)

attacked_sentences = df_neg_attacked[0].to_numpy()
unattacked_sentences = df_neg_unattacked[0].to_numpy()
attacked_labels = np.array(df_neg_attacked[1])
unattacked_labels = np.array(df_neg_unattacked[1])
'''
df_neg_attacked = pd.read_csv("/content/drive/MyDrive/MLCYBER/Data/Fake-News/fake_news_attacked.csv", header=None)
df_neg_unattacked = pd.read_csv("/content/drive/MyDrive/MLCYBER/Data/Fake-News/fake_news_unattacked.csv", header=None)
df_neg_attacked.columns = ['statement','label']
df_neg_unattacked.columns = ['statement','label']

attacked_sentences = df_neg_attacked['statement'].to_numpy()[1:]
unattacked_sentences = df_neg_unattacked['statement'].to_numpy()[1:]
attacked_labels = np.array(df_neg_attacked['label'][1:], dtype = np.int32)
unattacked_labels = np.array(df_neg_unattacked['label'][1:], dtype = np.int32)
'''

'\ndf_neg_attacked = pd.read_csv("/content/drive/MyDrive/MLCYBER/Data/Fake-News/fake_news_attacked.csv", header=None)\ndf_neg_unattacked = pd.read_csv("/content/drive/MyDrive/MLCYBER/Data/Fake-News/fake_news_unattacked.csv", header=None)\ndf_neg_attacked.columns = [\'statement\',\'label\']\ndf_neg_unattacked.columns = [\'statement\',\'label\']\n\nattacked_sentences = df_neg_attacked[\'statement\'].to_numpy()[1:]\nunattacked_sentences = df_neg_unattacked[\'statement\'].to_numpy()[1:]\nattacked_labels = np.array(df_neg_attacked[\'label\'][1:], dtype = np.int32)\nunattacked_labels = np.array(df_neg_unattacked[\'label\'][1:], dtype = np.int32)\n'

In [9]:
index = 102
print(unattacked_sentences[index])
print(attacked_sentences[index])

says congressman jon runyan does not want to cut taxes for the middle class, but only for millionaires.
says congressman jon runyan wants to cut taxes for the middle class, but only for millionaires.


In [ ]:
model.evaluate(attacked_sentences, attacked_labels)
model.evaluate(unattacked_sentences, unattacked_labels)

267/267 [==============================] - 115s 432ms/step - loss: 0.0275 - accuracy: 0.9943


[0.027486611157655716, 0.9942582845687866]

In [ ]:
predict_labels_unt = model(unattacked_sentences)
predict_labels_att = model(attacked_sentences)

In [ ]:
labels_unt = []
labels_att = []
for i in range(len(predict_labels_unt)):
  if predict_labels_unt[i][0] >= 0.5:
    labels_unt.append(1)
  else:
    labels_unt.append(0)
  if predict_labels_att[i][0] >= 0.5:
    labels_att.append(1)
  else:
    labels_att.append(0)
labels_unt = np.array(labels_unt)
labels_att = np.array(labels_att)

In [ ]:
total = 0
success = 0
for i in range(len(labels_unt)):
  if labels_unt[i] == unattacked_labels[i]:
    total += 1
    if labels_att[i] != attacked_labels[i]:
      success += 1
    #else:
      #print(attacked_sentences[i])
      #print(unattacked_sentences[i])
print(f"{success} out of the total {total} correct datas are attacked successfully")

8481 out of the total 8485 correct datas are attacked successfully


In [ ]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper
from textattack import AttackArgs
from textattack import Attacker
from textattack.datasets import Dataset

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1-logits, logits), dim=1)
        return final_preds

In [ ]:
from textattack.attack_recipes import DeepWordBugGao2018 as attack_recipe

i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
# attacker.attack_dataset()

textattack: Unknown if model of class <class 'load_model.lstm'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
